In [7]:
import os
import json
import pandas as pd
from pathlib import Path
import numpy as np
import ast
from tqdm import tqdm

# Data Extraction

In [37]:
ASJC_MAP = {
    '1000': 'Multidisciplinary',
    '1100': 'Agricultural and Biological Sciences',
    '1200': 'Arts and Humanities',
    '1300': 'Biochemistry, Genetics and Molecular Biology',
    '1303': 'Biochemistry',
    '1400': 'Business, Management and Accounting',
    '1500': 'Chemical Engineering',
    '1600': 'Chemistry',
    '1602': 'Analytical Chemistry',
    '1607': 'Spectroscopy',
    '1700': 'Computer Science',
    '1800': 'Decision Sciences',
    '1900': 'Earth and Planetary Sciences',
    '1909': 'Geotechnical Engineering and Engineering Geology',
    '2000': 'Economics, Econometrics and Finance',
    '2100': 'Energy',
    '2200': 'Engineering',
    '2208': 'Electrical and Electronic Engineering',
    '2209': 'Industrial and Manufacturing Engineering',
    '2211': 'Mechanics of Materials',
    '2300': 'Environmental Science',
    '2304': 'Environmental Chemistry',
    '2305': 'Environmental Engineering',
    '2311': 'Waste Management and Disposal',
    '2400': 'Immunology and Microbiology',
    '2500': 'Materials Science',
    '2504': 'Materials Chemistry',
    '2508': 'Surfaces, Coatings and Films',
    '2600': 'Mathematics',
    '2700': 'Medicine',
    '2739': 'Occupational Health',
    '2800': 'Neuroscience',
    '2900': 'Nursing',
    '3000': 'Pharmacology, Toxicology and Pharmaceutics',
    '3100': 'Physics and Astronomy',
    '3104': 'Condensed Matter Physics',
    '3110': 'Surfaces and Interfaces',
    '3200': 'Psychology',
    '3202': 'Applied Psychology',
    '3300': 'Social Sciences',
    '3400': 'Veterinary',
    '3500': 'Dentistry',
    '3600': 'Health Professions'
}

def get_scopus_id(itemidlist):
    """Safely extracts the Scopus ID."""
    try:
        if not itemidlist or 'itemid' not in itemidlist:
            return None
        for item in itemidlist.get('itemid', []):
            if item.get('@idtype') == 'SCP':
                return item.get('$')
    except Exception:
        return None
    return None

def get_authors(author_groups):
    """Extracts and de-duplicates author names."""
    try:
        authors = {}
        for group in author_groups:
            author_data = group.get('author', [{}])[0]
            auid = author_data.get('@auid')
            name = author_data.get('preferred-name', {}).get('ce:indexed-name')
            if auid and name:
                authors[auid] = name
        return "; ".join(authors.values())
    except Exception:
        return None

def get_affiliations(author_groups):
    """Extracts and de-duplicates unique organization names."""
    try:
        affil_set = set()
        for group in author_groups:
            affil = group.get('affiliation', {})
            org_data = affil.get('organization', {})
            org_name = None
            if isinstance(org_data, list):
                org_name = ", ".join([org.get('$') for org in org_data if org.get('$')])
            elif isinstance(org_data, dict):
                org_name = org_data.get('$')
            if org_name:
                affil_set.add(org_name)
        return "; ".join(affil_set)
    except Exception:
        return None

def get_asjc_raw(classificationgroup):
    """Safely extracts the raw ASJC data (could be a string or a list string)."""
    try:
        if not classificationgroup or 'classifications' not in classificationgroup:
            return np.nan
        
        classifications = classificationgroup.get('classifications', [])
        asjc_codes = []
        
        for classification in classifications:
            if classification.get('@type') == 'ASJC':
                asjc_codes.append(classification.get('classification'))
        
        if not asjc_codes:
            return np.nan
        
        
        if len(asjc_codes) == 1:
            return asjc_codes[0]
        
        
        list_of_dicts_str = str([{"$": code} for code in asjc_codes])
        return list_of_dicts_str

    except Exception:
        return np.nan

def translate_asjc(raw_asjc_value, code_map):
    if np.all(pd.isna(raw_asjc_value)):
        return np.nan
    
    s_value = str(raw_asjc_value).strip()
    codes_to_translate = []
    
    
    if s_value.startswith('['):
        try:
            list_data = ast.literal_eval(s_value)
            codes_to_translate = [item['$'] for item in list_data if isinstance(item, dict) and '$' in item]
        except Exception:
            return "Parsing Error"
    else:
        codes_to_translate = [s_value]
    
    translated_names = [code_map.get(code, f"Unknown Code ({code})") for code in codes_to_translate]
    
    return ", ".join(translated_names)

def process_folder(folder_path, output_csv):
    """
    Scans a folder for JSON files, extracts publication data,
    and saves it to a CSV file.
    """
    all_files_data = []  

    print(f"Scanning files in: {folder_path}")
    
    try:
        item_names = os.listdir(folder_path)
    except FileNotFoundError:
        print(f"Error: Folder not found at {folder_path}")
        print("Please update the 'folder_path' variable to the correct location.")
        return
    except Exception as e:
        print(f"Error listing files: {e}")
        return
 
    for item_name in item_names:
        file_path = os.path.join(folder_path, item_name)
        
        if not os.path.isfile(file_path):
            continue    
        
        if item_name == output_csv:
            continue
        
        if item_name == "extracted_publication_data.csv":
            continue

        try:
            
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            data = json.loads(content)
            
            item = data.get('abstracts-retrieval-response', {}).get('item', {})
            if not item:
                raise ValueError("Not a valid abstracts-retrieval-response file")
                
            bibrecord = item.get('bibrecord', {})
            head = bibrecord.get('head', {})
            item_info = bibrecord.get('item-info', {})
            source = head.get('source', {})
            chapter_title = head.get('citation-title')
            publication_year = source.get('publicationyear', {}).get('@first')
            book_title = source.get('sourcetitle')
            publisher = source.get('publisher', {}).get('publishername')
            doi = item_info.get('itemidlist', {}).get('ce:doi')
            scopus_id = get_scopus_id(item_info.get('itemidlist', {}))
            authors = get_authors(head.get('author-group', []))
            affiliations = get_affiliations(head.get('author-group', []))
            asjc_raw = get_asjc_raw(head.get('enhancement', {}).get('classificationgroup'))
            asjc_translation = translate_asjc(asjc_raw, ASJC_MAP)
            row_data = {
                "file_name": item_name,
                "chapter_title": chapter_title,
                "doi": doi,
                "scopus_id": scopus_id,
                "publication_year": publication_year,
                "book_title": book_title,
                "publisher": publisher,
                "authors": authors,
                "affiliation": affiliations,
                "ASJC": asjc_raw,  
                "ASJC_translation": asjc_translation 
            }
            
            all_files_data.append(row_data)
            print(f"Successfully processed: {item_name}")

        except json.JSONDecodeError:
            
            print(f"Skipping file: {item_name} (Not valid JSON content)")
        except Exception as e:
            
            print(f"Skipping file: {item_name} (Error: {e})")
            
    if all_files_data:
        df = pd.DataFrame(all_files_data)
        
        
        cols_order = [
            "file_name", "chapter_title", "doi", "scopus_id", 
            "publication_year", "book_title", "publisher", "authors", 
            "affiliation", "ASJC", "ASJC_translation"
        ]
        
        df = df[cols_order]
        
        df.to_csv(output_csv, index=False)
        
        print(f"\nSuccessfully processed {len(all_files_data)} file(s).")
        print(f"Data saved to '{output_csv}'")
        print("\nDataFrame Head:")
        print(df.head())
    else:
        print("No valid publication JSON files were found or processed.")

folder_to_scan = "C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/2018" # Folder Path

output_file_name = "extracted.csv" 


process_folder(folder_to_scan, output_file_name)

Scanning files in: C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/2018
Successfully processed: 201800000
Successfully processed: 201800001
Successfully processed: 201800002
Successfully processed: 201800003
Successfully processed: 201800004
Successfully processed: 201800005
Successfully processed: 201800006
Successfully processed: 201800007
Successfully processed: 201800008
Successfully processed: 201800009
Successfully processed: 201800010
Successfully processed: 201800011
Successfully processed: 201800012
Successfully processed: 201800013
Successfully processed: 201800014
Successfully processed: 201800015
Successfully processed: 201800016
Successfully processed: 201800017
Successfully processed: 201800018
Successfully processed: 201800019
Successfully processed: 201800020
Successfully processed: 201800021
Successfully processed: 201800022
Successfully processed: 201800023
Successfully processed: 201800024
Successfully processed: 201800025
Successfully processed: 20180002

KeyboardInterrupt: 

In [38]:
ASJC_MAP = {
    '1000': 'Multidisciplinary', '1100': 'Agricultural and Biological Sciences',
    '1200': 'Arts and Humanities', '1300': 'Biochemistry, Genetics and Molecular Biology',
    '1303': 'Biochemistry', '1400': 'Business, Management and Accounting',
    '1500': 'Chemical Engineering', '1600': 'Chemistry', '1602': 'Analytical Chemistry',
    '1607': 'Spectroscopy', '1700': 'Computer Science', '1800': 'Decision Sciences',
    '1900': 'Earth and Planetary Sciences', '1909': 'Geotechnical Engineering',
    '2000': 'Economics, Econometrics and Finance', '2100': 'Energy',
    '2200': 'Engineering', '2208': 'Electrical and Electronic Engineering',
    '2209': 'Industrial and Manufacturing Engineering', '2211': 'Mechanics of Materials',
    '2300': 'Environmental Science', '2304': 'Environmental Chemistry',
    '2305': 'Environmental Engineering', '2311': 'Waste Management and Disposal',
    '2400': 'Immunology and Microbiology', '2500': 'Materials Science',
    '2504': 'Materials Chemistry', '2508': 'Surfaces, Coatings and Films',
    '2600': 'Mathematics', '2700': 'Medicine', '2739': 'Occupational Health',
    '2800': 'Neuroscience', '2900': 'Nursing', '3000': 'Pharmacology',
    '3100': 'Physics and Astronomy', '3104': 'Condensed Matter Physics',
    '3110': 'Surfaces and Interfaces', '3200': 'Psychology', '3202': 'Applied Psychology',
    '3300': 'Social Sciences', '3400': 'Veterinary', '3500': 'Dentistry',
    '3600': 'Health Professions'
}

def get_nested(data, path, default=None):
    """Safely navigates nested dictionaries."""
    try:
        for key in path:
            data = data[key]
        return data
    except (KeyError, TypeError, AttributeError):
        return default

def get_scopus_id(itemidlist):
    try:
        if not itemidlist or 'itemid' not in itemidlist: return None
        for item in itemidlist.get('itemid', []):
            if item.get('@idtype') == 'SCP': return item.get('$')
    except: return None

def get_authors(author_groups):
    try:
        if not author_groups: return None
        if isinstance(author_groups, dict): author_groups = [author_groups]
        authors = {}
        for group in author_groups:
            alist = group.get('author', [])
            if isinstance(alist, dict): alist = [alist]
            for a in alist:
                if a.get('@auid') and a.get('preferred-name', {}).get('ce:indexed-name'):
                    authors[a['@auid']] = a['preferred-name']['ce:indexed-name']
        return "; ".join(authors.values())
    except: return None

def get_affiliations(author_groups):
    try:
        if not author_groups: return None
        if isinstance(author_groups, dict): author_groups = [author_groups]
        affil_set = set()
        for group in author_groups:
            affil = group.get('affiliation', {})
            org = affil.get('organization', {})
            if isinstance(org, list):
                name = ", ".join([o.get('$') for o in org if o.get('$')])
            elif isinstance(org, dict):
                name = org.get('$')
            else: name = None
            if name: affil_set.add(name)
        return "; ".join(affil_set)
    except: return None

def get_asjc_raw(classificationgroup):
    try:
        if not classificationgroup or 'classifications' not in classificationgroup: return np.nan
        clist = classificationgroup.get('classifications', [])
        if isinstance(clist, dict): clist = [clist]
        codes = [c.get('classification') for c in clist if c.get('@type') == 'ASJC']
        if not codes: return np.nan
        if len(codes) == 1: return codes[0]
        return str([{"$": c} for c in codes]) 
    except: return np.nan

def translate_asjc(raw_val, mapper):
    if np.all(pd.isna(raw_val)): return np.nan
    s = str(raw_val).strip()
    try:
        if s.startswith('['):
            data = ast.literal_eval(s)
            codes = [i['$'] for i in data if isinstance(i, dict) and '$' in i]
        else:
            codes = [s]
        return ", ".join([mapper.get(c, f"Unknown({c})") for c in codes])
    except: return "Error"

def get_abstract(head):
    try:
        data = head.get('abstracts')
        if isinstance(data, str): return data
        if isinstance(data, dict): return data.get('$', None)
    except: return None

def get_cover_date(source, coredata, process_info):
    """Tries multiple places to find a valid date."""
    try:
        
        if coredata and coredata.get('prism:coverDate'):
            return coredata.get('prism:coverDate')

        
        pd = source.get('publicationdate', {})
        if pd.get('year') and pd.get('month') and pd.get('day'):
            return f"{pd['year']}-{pd['month']}-{pd['day']}"
        
        
        sd = process_info.get('ait:date-sort', {})
        if sd.get('@year') and sd.get('@month') and sd.get('@day'):
            return f"{sd['@year']}-{sd['@month']}-{sd['@day']}"
        
        
        if pd.get('year'):
            return f"{pd['year']}-{pd.get('month', '01')}-01"
            
        return None
    except: return None

def get_author_keywords(root_data):
    """Extracts keywords from the root authkeywords block."""
    try:
        keywords = root_data.get('authkeywords', {}).get('author-keyword', [])
        if isinstance(keywords, dict): keywords = [keywords]
        return "; ".join([k.get('$', '') for k in keywords if '$' in k])
    except: return None

def process_folder_full_columns(root_path, output_csv):
    all_data = []
    files_processed = 0
    
    print(f"Scanning all files in: {root_path}")

    if not os.path.exists(root_path):
        print("Error: Folder path does not exist.")
        return

    for root, dirs, files in os.walk(root_path):
        for fname in tqdm(files, desc=f"Reading {os.path.basename(root)}"):
            if fname.startswith('.') or fname.endswith('.csv'): continue
            
            fpath = os.path.join(root, fname)
            
            try:
                with open(fpath, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                data = json.loads(content)
                root_response = data.get('abstracts-retrieval-response', {})
                item = root_response.get('item', {})
                coredata = root_response.get('coredata', {}) 
                
                if not item: continue
                
                bib = item.get('bibrecord', {})
                head = bib.get('head', {})
                source = head.get('source', {})
                info = bib.get('item-info', {})
                
                ct = head.get('citation-title')
                chapter_title = ct.get('titletext') if isinstance(ct, dict) else ct
                
                doi = info.get('itemidlist', {}).get('ce:doi')
                scopus_id = get_scopus_id(info.get('itemidlist', {}))
                
                pub_year = source.get('publicationyear', {}).get('@first')
                book_title = source.get('sourcetitle')
                publisher = source.get('publisher', {}).get('publishername')
                
                authors = get_authors(head.get('author-group', []))
                affiliation = get_affiliations(head.get('author-group', []))
                
                asjc_raw = get_asjc_raw(head.get('enhancement', {}).get('classificationgroup'))
                asjc_trans = translate_asjc(asjc_raw, ASJC_MAP)
                
                abstract = get_abstract(head)
                cover_date = get_cover_date(source, coredata, item.get('ait:process-info', {}))
                description = coredata.get('dc:description')
                agg_type = coredata.get('prism:aggregationType')
                auth_keywords = get_author_keywords(root_response)
                ref_count = bib.get('tail', {}).get('bibliography', {}).get('@refcount')
                row = {
                    "file_name": fname,
                    "chapter_title": chapter_title,
                    "doi": doi,
                    "scopus_id": scopus_id,
                    "publication_year": pub_year,
                    "book_title": book_title,
                    "publisher": publisher,
                    "authors": authors,
                    "affiliation": affiliation,
                    "ASJC": asjc_raw,
                    "ASJC_translation": asjc_trans,
                    "abstract": abstract,
                    "cover_date": cover_date,        
                    "description": description,      
                    "aggregation_type": agg_type,    
                    "author_keywords": auth_keywords,
                    "reference_count": ref_count     
                }
                all_data.append(row)
                files_processed += 1

            except json.JSONDecodeError:
                pass 
            except Exception:
                pass 
    
    if all_data:
        df = pd.DataFrame(all_data)
        cols = [
            "file_name", "chapter_title", "doi", "scopus_id", 
            "publication_year", "cover_date", 
            "book_title", "publisher", "aggregation_type",
            "authors", "affiliation", 
            "abstract", "description", "author_keywords",
            "ASJC", "ASJC_translation", "reference_count"
        ]
        
        cols = [c for c in cols if c in df.columns]
        df = df[cols]
        
        df.to_csv(output_csv, index=False)
        print(f"\nDone! Processed {files_processed} valid papers.")
        print(f"Data saved to: {output_csv}")
        print(df.head(2))
    else:
        print("\nNo valid Scopus JSON data found.")

target_folder = "C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/"
output_filename = "extracted_data.csv"

process_folder_full_columns(target_folder, output_filename)

🚀 Scanning all files in: C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/


Reading 2023: 100%|██████████| 2890/2890 [01:53<00:00, 25.44it/s]



Done! Processed 19805 valid papers.
Data saved to: extracted_data.csv
   file_name                                      chapter_title  \
0  201800000  Public health and international epidemiology f...   
1  201800001  Flexible Printed Active Antenna for Digital Te...   

                            doi    scopus_id publication_year  cover_date  \
0  10.1007/978-3-319-98485-8_15  85077976956             2018  2018-12-31   
1   10.23919/PIERS.2018.8597669  85060936020             2018  2018-12-31   

                                          book_title  \
0  Radiology in Global Health: Strategies, Implem...   
1    Progress in Electromagnetics Research Symposium   

                                           publisher       aggregation_type  \
0                  Springer International Publishing                   Book   
1  Institute of Electrical and Electronics Engine...  Conference Proceeding   

                       authors  \
0   Pongpirul K.; Lungren M.P.   
1  Pratumsiri T.; Ja

In [39]:
df = pd.read_csv("extracted_data.csv")
df

,file_name,chapter_title,doi,scopus_id,publication_year,cover_date,book_title,publisher,aggregation_type,authors,affiliation,abstract,description,author_keywords,ASJC,ASJC_translation,reference_count
0,201800000,Public health and international epidemiology f...,10.1007/978-3-319-98485-8_15,85077976956,2018,2018-12-31,"Radiology in Global Health: Strategies, Implem...",Springer International Publishing,Book,Pongpirul K.; Lungren M.P.,Bumrungrad International Hospital; Internation...,NaN,NaN,NaN,2700,Medicine,76
1,201800001,Flexible Printed Active Antenna for Digital Te...,10.23919/PIERS.2018.8597669,85060936020,2018,2018-12-31,Progress in Electromagnetics Research Symposium,Institute of Electrical and Electronics Engine...,Conference Proceeding,Pratumsiri T.; Janpugdee P.,"Department of Electrical Engineering, Wireless...","© 2018 The Institute of Electronics, Informati...",This paper presents the development of a flexi...,NaN,"[{'$': '2208'}, {'$': '2504'}]","Electrical and Electronic Engineering, Materia...",4
2,201800002,Parametric study of hydrogen production via so...,10.1016/j.ces.2018.08.042,85052201238,2018,2018-12-31,Chemical Engineering Science,Elsevier Ltd,Journal,Phuakpunk K.; Assabumrungrat S.; Chalermsinsuw...,"Department of Mechanical Engineering, Faculty ...",© 2018 Elsevier LtdComputational fluid dynamic...,Computational fluid dynamics was applied for s...,Circulating fluidized bed; Computational fluid...,"[{'$': '1600'}, {'$': '1500'}, {'$': '2209'}]","Chemistry, Chemical Engineering, Industrial an...",42
3,201800003,Superhydrophobic coating from fluoroalkylsilan...,10.1016/j.apsusc.2018.08.059,85051498032,2018,2018-12-31,Applied Surface Science,Elsevier B.V.,Journal,Saengkaew J.; Le D.; Samart C.; Kongparakul S....,"Department of Frontier Materials Chemistry, Gr...",© 2018 Elsevier B.V. A superhydrophobic/supero...,A superhydrophobic/superoleophilic mesh was su...,Encapsulation; Fluoroalkylsilane; Natural rubb...,"[{'$': '1600'}, {'$': '3104'}, {'$': '3100'}, ...","Chemistry, Condensed Matter Physics, Physics a...",45
4,201800004,Electrochemical impedance-based DNA sensor usi...,10.1016/j.aca.2018.07.045,85050678366,2018,2018-12-31,Analytica Chimica Acta,Elsevier B.V.,Journal,Teengam P.; Siangproh W.; Tuantranont A.; Vila...,"Department of Chemistry, Colorado State Univer...",© 2018 Elsevier B.V. A label-free electrochemi...,A label-free electrochemical DNA sensor based ...,acpcPNA; Electrochemical impedance spectroscop...,"[{'$': '1602'}, {'$': '1303'}, {'$': '2304'}, ...","Analytical Chemistry, Biochemistry, Environmen...",55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,202302885,Long-chain bio-olefins production via oxidativ...,10.1016/j.cattod.2021.07.034,85111945558,2023,2023-01-01,Catalysis Today,Elsevier B.V.,Journal,Le D.; Hinchiranan N.; Chaidherasuwet N.; Ruea...,"Program in Petrochemistry and Polymer Science,...",© 2021 Elsevier B.V.Long-chain α-olefins (≥ C1...,Long-chain α-olefins (≥ C10) are normally appl...,Long-chain olefins; Mesoporous KIT-6; Oleic ac...,"[{'$': '1503'}, {'$': '1600'}]","Unknown(1503), Chemistry",63
19801,202302886,Recent Developments and Applications of Microf...,10.1080/10408347.2021.1949695,85111408415,2023,2023-01-01,Critical Reviews in Analytical Chemistry,Taylor and Francis Ltd.,Journal,Alahmad W.; Varanusupakul P.; Varanusupakul P.,"Department of Chemistry, Faculty of Science, C...","© 2021 Taylor & Francis Group, LLC.Nowadays, f...","Nowadays, food safety has become a major conce...",Biological hazards; chemical hazards; food con...,1602,Analytical Chemistry,115
19802,202302887,"Social justice, education and peacebuilding: c...",10.1080/03057925.2021.1951666,85110903700,2023,2023-01-01,Compare,Routledge,Journal,Pherali T.,Centre for Education and International Develop...,© 2021 The Author(s). Published by Informa UK ...,Education is increasingly becoming central to ...,conflict; Education; peacebuilding; social jus...,3304,Unknown(3304

# Data Preprocessing

In [40]:
cols = [
    "chapter_title",
    "abstract",
    "author_keywords",
    "description",
    "book_title",
    "publication_year",
    "ASJC_translation",
]

df_clean = df[cols].copy()

text_cols = [
    "chapter_title",
    "abstract",
    "author_keywords",
    "book_title"
]

# Fill missing text
df_clean[text_cols] = df_clean[text_cols].fillna("")

# Fill numbers
df_clean["publication_year"] = df_clean["publication_year"].fillna(-1)

# Remove rows with all empty text
df_clean = df_clean[
    (df_clean["chapter_title"] != "") |
    (df_clean["abstract"] != "") |
    (df_clean["author_keywords"] != "")
]

# Remove duplicates
df_clean = df_clean.drop_duplicates(subset=["chapter_title", "abstract"])

import re

def remove_copyright_year(text):
    if not isinstance(text, str):
        return text
    
    # Remove pattern like: "© 2021", "©2021", "© 1999"
    return re.sub(r'^©\s*\d{4}\s*', '', text)

df_clean["abstract_clean"] = df_clean["abstract"].apply(remove_copyright_year)
df_clean.drop(columns=["abstract"], inplace=True)

df_clean.dropna(inplace=True)

In [41]:
df_clean

,chapter_title,author_keywords,description,book_title,publication_year,ASJC_translation,abstract_clean
1,Flexible Printed Active Antenna for Digital Te...,,This paper presents the development of a flexi...,Progress in Electromagnetics Research Symposium,2018,"Electrical and Electronic Engineering, Materia...","The Institute of Electronics, Information and ..."
2,Parametric study of hydrogen production via so...,Circulating fluidized bed; Computational fluid...,Computational fluid dynamics was applied for s...,Chemical Engineering Science,2018,"Chemistry, Chemical Engineering, Industrial an...",Elsevier LtdComputational fluid dynamics was a...
3,Superhydrophobic coating from fluoroalkylsilan...,Encapsulation; Fluoroalkylsilane; Natural rubb...,A superhydrophobic/superoleophilic mesh was su...,Applied Surface Science,2018,"Chemistry, Condensed Matter Physics, Physics a...",Elsevier B.V. A superhydrophobic/superoleophil...
4,Electrochemical impedance-based DNA sensor usi...,acpcPNA; Electrochemical impedance spectroscop...,A label-free electrochemical DNA sensor based ...,Analytica Chimica Acta,2018,"Analytical Chemistry, Biochemistry, Environmen...",Elsevier B.V. A label-free electrochemical DNA...
5,Evaluation of outsourcing transportation contr...,Design of experiment; Optimal fleet size; Outs...,This paper aims to develop an approach to iden...,Polish Journal of Management Studies,2018,"Unknown(1403), Unknown(1408), Unknown(1407)",", Czestochowa University of Technology. All ri..."
...,...,...,...,...,...,...,...
19800,Long-chain bio-olefins production via oxidativ...,Long-chain olefins; Mesoporous KIT-6; Oleic ac...,Long-chain α-olefins (≥ C10) are normally appl...,Catalysis Today,2023,"Unknown(1503), Chemistry",Elsevier B.V.Long-chain α-olefins (≥ C10) are ...
19801,Recent Developments and Applications of Microf...,Biological hazards; chemical hazards; food con...,"Nowadays, food safety has become a major conce...",Critical Reviews in Analytical Chemistry,2023,Analytical Chemistry,"Taylor & Francis Group, LLC.Nowadays, food saf..."
19802,"Social justice, education and peacebuilding: c...",conflict; Education; peacebuilding; social jus...,Education is increasingly becoming central to ...,Compare,2023,Unknown(3304),The Author(s). Published by Informa UK Limited...
19803,Effects of black soldier fly (Hermetia illucen...,Anabas testudineus; Black soldier fly; fish me...,The effects of replacing fish meal protein wit...,Journal of Applied Aquaculture,2023,"Unknown(2303), Unknown(1104)",Taylor & Francis.The effects of replacing fish...


In [42]:
df_clean.isnull().sum()

chapter_title       0
author_keywords     0
description         0
book_title          0
publication_year    0
ASJC_translation    0
abstract_clean      0
dtype: int64

# Others

In [24]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

class TrendGeneratorAI:
    def __init__(self, dataframe):
        self.df = dataframe.copy()
        # Pre-fill NaNs to prevent errors
        text_cols = ['chapter_title', 'abstract', 'author_keywords', 'ASJC_translation']
        for col in text_cols:
            if col in self.df.columns:
                self.df[col] = self.df[col].fillna('')

    def clean_text(self, text):
        """Basic text cleaning to help the AI understand."""
        if not isinstance(text, str): 
            return ""
        # Lowercase and remove special characters
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text) 
        return text

    def generate_trends(self, category_input, start_year, end_year):
        """
        AI Logic:
        1. Filters data by user input.
        2. Vectorizes text (TF-IDF).
        3. Uses NMF (Generative Topic Modeling) to extract top 3 themes.
        """
        print(f"\n--- 🤖 AI Status: Initializing scan for '{category_input}' ({start_year}-{end_year})... ---")

        # 1. FILTER DATA
        # We use str.contains because a paper might belong to multiple categories
        mask_category = self.df['ASJC_translation'].str.contains(category_input, case=False, na=False)
        mask_year = (self.df['publication_year'] >= start_year) & (self.df['publication_year'] <= end_year)
        
        subset = self.df[mask_category & mask_year].copy()
        
        # Check if we have enough data
        if len(subset) < 5:
            return [f"⚠️ Not enough data ({len(subset)} papers) to generate trends. Try a wider year range."]

        print(f"--- 📚 AI Status: Analyzing {len(subset)} publications... ---")

        # 2. PREPARE TEXT FOR AI
        # We combine Title (x3 weight), Keywords (x2 weight), and Abstract
        # This teaches the AI that Titles are the most important part of the trend.
        subset['ai_text'] = (
            (subset['chapter_title'] + " ") * 3 + 
            (subset['author_keywords'] + " ") * 2 + 
            subset['abstract']
        ).apply(self.clean_text)

        # 3. TF-IDF VECTORIZATION (Math translation of words)
        # ignore words that appear in >95% of docs (too common) or <2 docs (too rare)
        tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
        
        try:
            tfidf = tfidf_vectorizer.fit_transform(subset['ai_text'])
        except ValueError:
            return ["⚠️ Text data is too sparse to generate trends."]

        # 4. NMF MODEL (The 'Generative' Engine)
        # We ask the model to generate exactly 3 latent topics
        n_topics = 3
        nmf_model = NMF(n_components=n_topics, random_state=42, init='nndsvd')
        nmf_model.fit(tfidf)

        # 5. DECODE OUTPUT
        feature_names = tfidf_vectorizer.get_feature_names_out()
        trends = []

        for topic_idx, topic in enumerate(nmf_model.components_):
            # Get the top 4 words that define this trend
            top_indices = topic.argsort()[:-5:-1]
            top_words = [feature_names[i] for i in top_indices]
            
            # Format nicely as a "Title Trend"
            trend_title = " ".join([word.capitalize() for word in top_words])
            trends.append(f"Trend {topic_idx + 1}: {trend_title}")

        return trends

# ==========================================
# 🚀 MAIN EXECUTION BLOCK
# ==========================================

# 1. Load your DataFrame (assuming 'df' is already loaded in your environment)
# If not, uncomment the line below:
# df = pd.read_csv('extracted_publications_with_translation.csv')

# 2. Initialize the AI Engine
ai_engine = TrendGeneratorAI(df)

# 3. USER INPUT SIMULATION
# In a real app, these would come from input boxes
user_category = input("Enter Category (e.g., Medicine, Engineering, Chemistry): ")
try:
    user_start_year = int(input("Enter Start Year (e.g., 2018): "))
    user_end_year = int(input("Enter End Year (e.g., 2021): "))
except ValueError:
    print("Please enter valid numbers for years.")
    user_start_year = 2018
    user_end_year = 2023

# 4. RUN GENERATION
generated_results = ai_engine.generate_trends(user_category, user_start_year, user_end_year)

# 5. DISPLAY RESULTS
print("\n" + "="*40)
print(f"🔥 GENERATED TITLE TRENDS ({user_category})")
print("="*40)
for result in generated_results:
    print(result)
print("="*40)


--- 🤖 AI Status: Initializing scan for 'Medicine' (2016-2018)... ---
--- 📚 AI Status: Analyzing 84 publications... ---

🔥 GENERATED TITLE TRENDS (Medicine)
Trend 1: Adverse Events Incidents Perioperative
Trend 2: Patients Mortality Clinical Study
Trend 3: Pain Version Thai Reliability
